# Social DistilBERT Model Using Single Batch 1 + Batch 2

In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed"
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [2]:
# import os

# # Define the folder path and themes
# folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method'
# themes = [
#     'Aspirational', 'Attainment', 'Community Consciousness', 'Familial', 'Filial Piety', 
#     'First Gen', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
# ]

# # Initialize an empty dictionary to store DataFrames
# batch_1_theme_dataframes = {}
# # Loop through each theme and load its corresponding file
# for theme in themes:
#     # Construct the filename without modifying the theme name
#     file_name = f"{theme}_sentence_level_batch_1_jaccard.csv"
#     file_path = os.path.join(folder_path, file_name)
    
#     # Check if the file exists before attempting to load
#     if os.path.exists(file_path):
#         batch_1_theme_dataframes[theme] = pd.read_csv(file_path)
#         print(f"Loaded {file_name}")
#     else:
#         print(f"File not found for theme: {theme}")

# # Define the folder path and themes
# folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method/batch_2'
# themes = [
#     'Aspirational', 'Attainment', 'Community Consciouss', 'Familial', 'Filial Piety', 
#     'First Generation', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
# ]

# # Initialize an empty dictionary to store DataFrames
# batch_2_theme_dataframes = {}
# # Loop through each theme and load its corresponding file
# for theme in themes:
#     # Construct the filename without modifying the theme name
#     file_name = f"{theme}_sentence_level_batch_2_jaccard.csv"
#     file_path = os.path.join(folder_path, file_name)
    
#     # Check if the file exists before attempting to load
#     if os.path.exists(file_path):
#         batch_2_theme_dataframes[theme] = pd.read_csv(file_path)
#         print(f"Loaded {file_name}")
#     else:
#         print(f"File not found for theme: {theme}")

In [3]:
merged_social_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/corrections/Social/Social_merged_Reevaluated.csv")


merged_social_df

,sentence,label,phrase
0,"i am here because i want to better myself my family, not only financially but in health.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
1,being in this instituion will pave a way for me to become a professional.,0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
2,i know that as a child i never thought of education and a career for someone who is undocumented.,0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
3,"i hope to reach a position in which i can inspire and prove to all of the ""immigrants"" in the usa that achieving and surpassing struggle is possible.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
4,"also, statistics have shown that people with higher education and wealth tend to be healthier.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
...,...,...,...
3272,"when i was younger, my mother was diagnosed with a brain tumor and it took months of convincing her to finally get surgery to remove the tumor.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolonging long and healthy lives.""]"
3273,"after experiencing those years of watching her suffer, i've developed a love for the medical field and knew i wanted to be apart of changing the lives of others through medicine.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolonging long and healthy lives.""]"
3274,"also, i've been granted the gift of imagination which allows me to write mini books, poems, and short stories, along with a long list of other talents.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolongi

In [4]:
# Find duplicates in the 'sentence' column
duplicate_texts = merged_social_df[merged_social_df.duplicated(subset='sentence', keep=False)]

# Display duplicates
duplicate_texts

,sentence,label,phrase
16,"i am here for many reasons: make myself proud happy getting education is a priveledge, knowledge is power!",0,['To help my community; I want to give back to all who have touched me.']
17,lowkey school makes me happy.,0,['To help my community; I want to give back to all who have touched me.']
18,to help my community; i want to give back to all who have touched me.,1,['To help my community; I want to give back to all who have touched me.']
19,"by going to school teaching this class, i am able to gain perspectives sufficient knowledge to provide a helping hand to all.",1,['To help my community; I want to give back to all who have touched me.']
20,"for change, i am here to change my life others.",0,['To help my community; I want to give back to all who have touched me.']
...,...,...,...
3043,it's funny because ive never been happier than i ever been because of everything ive been doing lately.,0,['Im in college because I know that I can make a change in my community. That everything I do with my life will not only help my family but everyone in my community. I have a drive to do better in a world that constantly belittles my community.The only way that Im able to create a better future for my community is by getting a PhD and create data dedicated in my community and create policy targeted towards my community. I know that I still have years until I finish my life academically but I know that I want to do everything to help my community.']
3044,i just know this is what i want to do with my life.,0,['Im in college because I know that I can make a change in my community. That everything I do with my life will not only help my family but everyone in my community. I have a drive to do better in a world that constantly belittles my community.The only way that Im able to create a better future for my community is by getting a PhD and create data dedicated in my community and create policy targeted towards my community. I know that I still have years until I finish my life academically but I know that I want to do everything to help my community.']
3045,i am here because i want to get a degree.,0,['Going to college will let you network with people and make good connections which can go a long way. I think that college is a great place to be as it is a melting pot of people and culture.']
3213,i am here to succeed.,0,"['', ""I am a kinesiology major so I am here to learn to help others. It's something that I have always felt was a part of me."", '']"


In [5]:
# Remove duplicates from the 'sentence' column
merged_social_df = merged_social_df.drop_duplicates(subset='sentence', keep='first')

# Display the resulting DataFrame
merged_social_df

,sentence,label,phrase
0,"i am here because i want to better myself my family, not only financially but in health.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
1,being in this instituion will pave a way for me to become a professional.,0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
2,i know that as a child i never thought of education and a career for someone who is undocumented.,0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
3,"i hope to reach a position in which i can inspire and prove to all of the ""immigrants"" in the usa that achieving and surpassing struggle is possible.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
4,"also, statistics have shown that people with higher education and wealth tend to be healthier.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']"
...,...,...,...
3272,"when i was younger, my mother was diagnosed with a brain tumor and it took months of convincing her to finally get surgery to remove the tumor.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolonging long and healthy lives.""]"
3273,"after experiencing those years of watching her suffer, i've developed a love for the medical field and knew i wanted to be apart of changing the lives of others through medicine.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolonging long and healthy lives.""]"
3274,"also, i've been granted the gift of imagination which allows me to write mini books, poems, and short stories, along with a long list of other talents.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolongi

In [6]:
# Shuffle the merged dataset
merged_social_df = shuffle(merged_social_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_social_df, test_size=0.1, random_state=18, stratify=merged_social_df["label"])


training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)


In [8]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (2664, 3) 
Test dataset shape: (296, 3)
Positive labels present in the dataset : 671  out of 2664 or 25.18768768768769%
Positive labels present in the test dataset : 75  out of 296 or 25.33783783783784%


In [9]:
training_df

,sentence,label,phrase
0,i really am proud on the decision i made because moving to a new city really helped me evolved as a person.,1,['I decided to come to San Francisco State University because I wanted to experience a new community.']
1,sf state is the only other school i got into other then whittier because of a transcript screw up at my high school.,0,"['I am in school because I seek to build a foundation of knowledge in my interests which range from English, Philosophy, and now Political Science and Astronomy. I seek a foundation in these subjects because I would love to become a teacher to do return the favor people have done for me in enlightening and opening my mind up to enjoying learning.']"
2,i decided to take sci 111 because i struggled with physics in high school.,0,"['I am someone who loves interacting with people, and I like working in groups. I feel like it is much easier to find a solution to a problem if lots of minds are put together. Sci provides the opportunity to interact with a diverse group of students who all come from different backgrounds, who all bring different perspectives and ideas.']"
3,"most importantly, as part of an underrepresented community i want my patients to feel comfortable by having a provider that gives care that is culturally and linguistically sensitive.",1,"['Most importantly, as part of an underrepresented community I want my patients to feel comfortable by having a provider that gives care that is culturally and linguistically sensitive.']"
4,i am looking forward to learn more of this subject and to be able to apply that in my future endeavors.,0,['This class is also an opportunity to meet and engage in conversation with my fellow scientist who also have to probably take this class because of their chosen major but also being able to get that insight from their own perspective on the material.']
...,...,...,...
2659,"also, i am here for myself knowing the life i want to live and that i'm going to chase my dreams staying dedicated.",0,['Learning about that and being able to help another person life is success and providing for the community around.']
2660,i am taking this sci class in order to learn more about organic chemistry.,0,['It is also a great class to make friends or meet new people that are also taking the same subject as you.']
2661,i also hope this class will help me understand physics more by just practicing more problems.,1,['I also want to meet new people so we can all study together and help each other']
2662,"for me personally, i am here to learn more about stem so i can gain the tools to equip me for medical school.",0,['mainly I want to make a difference in the medical community and use my skills to help others.']


In [10]:
test_df

,sentence,label,phrase
0,that's the kind of legacy i want to be remembered as and a future i want to leave for the children of tomorrow.,0,"[""It didn't matter if it was a world wide change or just making a difference in your community, as long as you were making a significant change to your community, that's what mattered the most. If everyone came together, learned about each other and have an open mind, the world would be in a better place.I, myself can't do much as a person at the moment, but my goal in life is to be in a position, where I can make a change and be an example to others.""]"
1,why am i in college?,0,"['With this class, I not only hope to get as much help as I can but also to meet people in order to make a study group and make connections for the future.']"
2,"since physics is basically another math class, i knew i would need to take extra steps to ensure my success.",0,"['I would be able to connect more with my fellow classmates. I figured that if I felt like I created a little community with the supplemental course, I would feel more comfortable with physics problems.']"
3,"i just know that since i am here, i need to make the most out of my time.",0,"['In a different and more general sense of why I am here, well, I know that I am here because I have given this gift of life and I try my best to make the best of it everyday.', 'That being said, I have came across various situations where peoples lives have been taken too early on and because of those experiences, I know how important it is to value living and being here.']"
4,"i want to fill people around me with positive memories, love, knowledge, and teachable moments.",0,"['I want to fill people around me with positive memories, love, knowledge, and teachable moments.']"
...,...,...,...
291,i like to know how there is so many different roads to take for a certain topic by just speaking to others.,0,"['I know the reason why everyone is in the class because they have similar reasons like I do; being able to connect with one another and build more information together. It gives me a confidence boost within myself to explore more, and try to make more friends. I realized the more people you know the extra help you can get with the sections I am struggling with, because someone is bound to know that topic by the heart or know a different way to explain. That is why I have been taking many SCI classes is because it gives me an opportunity to learn by others and make friends with different people.']"
292,i teach and learn lessons as i go as to what my purpose is.,1,"[""I am a rather spiritual person I would say and I am also a curious one and for that reason I've asked a lot of different people a lot of different questions. I'm not worried about the what happens next either because at the end of the day it is inevitable, and i try not to think about the what i'm supposed to do for now as much because it can really put you in a helpless state, so i'm just trying to trust my intuition and flow to where life takes me.""]"
293,i am here in this lab class because i needed to finish one of my general education requirements.,0,['Its also fun getting to collaborate with the other students in the class.']
294,many think that but i think otherwise.,0,['Going to college will let you network with people and make good connections which can go a long way. I think that college is a great place to be as it is a melting pot of people and culture.']


In [12]:
# Filter rows where the 'sentence' column contains a specific substring
substring = "i want to fill people around me with positive memories, love, knowledge, and teachable moments.	0"  # Replace with your desired substring
filtered_df = training_df[training_df['sentence'].str.contains(substring, case=False, na=False)]

# Display the filtered DataFrame
filtered_df

,sentence,label,phrase


In [32]:
# from sklearn.utils import resample
# import pandas as pd

# # Assuming `training_df` already exists and has a 'label' column

# # Check original class distribution
# print("Original class distribution:")
# print(training_df['label'].value_counts())

# # Set target distribution
# target_minority_ratio = 0.40  # 20%
# target_majority_ratio = 0.60  # 80%

# # Separate majority and minority classes
# minority_class = training_df[training_df['label'] == 1]
# majority_class = training_df[training_df['label'] == 0]

# # Use all samples from the minority class
# target_minority_count = len(minority_class)

# # Calculate the maximum possible number of samples for the majority class
# target_majority_count = min(len(majority_class), int(target_minority_count / target_minority_ratio * target_majority_ratio))

# # Debug information
# print(f"Minority samples: {target_minority_count}, "
#       f"Target majority count: {target_majority_count}, "
#       f"Available majority samples: {len(majority_class)}")

# # Perform undersampling
# if target_majority_count < len(majority_class):
#     undersampled_majority = resample(
#         majority_class,
#         replace=False,  # without replacement
#         n_samples=target_majority_count,
#         random_state=42
#     )
# else:
#     print("Target majority count exceeds available samples. Using all majority samples.")
#     undersampled_majority = majority_class

# # Combine the resampled majority class with the full minority class
# training_df = pd.concat([minority_class, undersampled_majority]).sample(frac=1, random_state=42).reset_index(drop=True)

# # Check new class distribution
# print("New class distribution:")
# print(training_df['label'].value_counts())

Original class distribution:
label
0    3864
1     848
Name: count, dtype: int64
Minority samples: 848, Target majority count: 1272, Available majority samples: 3864
New class distribution:
label
0    1272
1     848
Name: count, dtype: int64


In [ ]:
# Filter rows where the 'sentence' column contains a specific substring
substring = "i am here to learn"  # Replace with your desired substring
filtered_df = merged_df[merged_df['sentence'].str.contains(substring, case=False, na=False)]

# Display the filtered DataFrame
filtered_df

In [5]:
from sklearn.utils import resample
import pandas as pd

def oversample_minority(training_df, minority_label, majority_label, target_minority_ratio):
    """
    Oversample the minority class to achieve a desired class ratio.

    Parameters:
        training_df (pd.DataFrame): The input training dataset containing a 'label' column.
        minority_label (int): The label of the minority class (e.g., 1).
        majority_label (int): The label of the majority class (e.g., 0).
        target_minority_ratio (float): The desired ratio of the minority class (e.g., 0.18 for 18%).

    Returns:
        pd.DataFrame: The balanced dataset with oversampled minority class.
    """
    # Separate majority and minority classes
    minority_class = training_df[training_df['label'] == minority_label]
    majority_class = training_df[training_df['label'] == majority_label]

    # Calculate the target size for the minority class
    target_minority_count = int(len(majority_class) * (target_minority_ratio / (1 - target_minority_ratio)))

    # Debug information
    print(f"Minority samples: {len(minority_class)}, "
          f"Target minority count: {target_minority_count}, "
          f"Available majority samples: {len(majority_class)}")

    # Perform oversampling
    oversampled_minority = resample(
        minority_class,
        replace=True,  # with replacement
        n_samples=target_minority_count,
        random_state=seed
    )

    # Combine the resampled minority class with the majority class
    balanced_training_df = pd.concat([oversampled_minority, majority_class]).sample(frac=1, random_state=42).reset_index(drop=True)

    # Check the new class distribution
    print("New class distribution:")
    print(balanced_training_df['label'].value_counts())

    return balanced_training_df

In [6]:
train_df, validate_df = train_test_split(training_df, test_size=0.1, random_state=18)

In [22]:
# Assuming `training_df` is your dataset
training_df = oversample_minority(
    training_df=train_df,
    minority_label=1,  # The label of the minority class
    majority_label=0,  # The label of the majority class
    target_minority_ratio=0.30  # Target ratio for the minority class
)

Minority samples: 257, Target minority count: 1490, Available majority samples: 3478
New class distribution:
label
0    3478
1    1490
Name: count, dtype: int64


In [20]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))
pos_labels = len([n for n in validate_df['label'] if n==1])
print("Positive labels present in the validation dataset : {}  out of {} or {}%".format(pos_labels, len(validate_df['label']), (pos_labels/len(validate_df['label']))*100))

Training dataset shape: (2949, 3) 
Test dataset shape: (328, 3)
Positive labels present in the dataset : 731  out of 2949 or 24.788063750423873%
Positive labels present in the test dataset : 81  out of 328 or 24.695121951219512%


NameError: name 'validate_df' is not defined

In [24]:
print(training_df.shape)
print(test_df.shape)

(4968, 3)
(462, 3)


## 2. Experimental Design

In [25]:
validate_df

,sentence,label,phrase
1486,"i want to at least help animals, such as my bird and reptiles that need care, as there are not that many exotic vets, only primarily those who focus on cats and dogs.",0,"['I know I want to become an exotic Veterinarian, but I feel like I am not smart enough to get into Vet School. I want to at least help animals, such as my bird and reptiles that need care, as there are not that many exotic vets, only primarily those who focus on cats and dogs. If that is the path I eventually do not take, I am very interested in virology and would like to study them. Maybe that is why I am here, to at least provide a slight change to science, although it may not be a big one at least it is something I can contribute to the world.']"
2888,as i mentioned earlier im here because i need to take physics in as a course for my kinesiology degree.,0,['As I mentioned earlier Im here because I need to take physics in as a course for my Kinesiology degree. Another reason why I feel like Im here is to further my education and career . To keep striving at being a better person each and every day .']
4042,it's funny because ive never been happier than i ever been because of everything ive been doing lately.,0,['The only way that Im able to create a better future for my community is by getting a PhD']
2915,"science has always intrigued me, but i feel like the schooling system of stem classes kind of makes it harder for me to enjoy.",0,"[""In the context of school, is I am here for the reason I'm on the path to become a doctor.""]"
4081,"while in high school i was deciding between 3 school, but ultimately picked here.",0,"['I also wanted to go into civil engineer, and this was the only school with it, the others had mechanical but not civil. As to why I am currently here in this moment in time of writing this essay thing, I am a third year trying to become a civil engineer.']"
...,...,...,...
2402,"point and case, my cat sitting on my printer glaring at me because he wants something from me.",0,['This could help me simply better understand every day experiences but ultimately will help me in my future career as a physical therapist. Im at SFSU because I want a degree.. I want a degree in kinesiology because I want to pursue becoming a physical therapist.']
1107,"this class is also along side a lecture, which has been able to help me better understand concepts by doing experiments.",0,"['By the end of this class, I hope to have a better understanding of what physics is and that it will help me when I take my MCAT and enter medical school in the next couple of years after I graduate with my fouryear degree at San Francisco State.']"
1308,i am part of the first generation in my family to go to college and hopefully get a degree and i am trying to make my parents proud but also learn things for myself and get a career that will give me financial stability and also a career where i will help tons of people.,0,"['Why am I here, I am here because I want to become a doctor and go onto medical school. I am here to hopefully pass this class and come out with a good grade and complete the requirement in my major. I am here to get a degree to go on and become a doctor which is what I am studying for. I am part of the first generation in my family to go to college and hopefully get a degree and I am trying to make my parents proud but also learn things for myself and get a career that will give me financial stability and also a career where I will help tons of people.']"
3843,it may be helpful as future reference when there might be a situation or problem i may encounter in my life since some of the analysis and theories are relatable to use.,0,['I chose to be in this class because it is a requirement to fulfill in order for me to obtain my Cell and Molecular Biology bachelors degree.']


In [26]:
# MAXLEN = 256

# X_train = training_df['sentence']
# y_train = training_df['label']
# X_test = validate_df['sentence']
# y_test = validate_df['label']
# # Split the data
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18, stratify=y)
# # X_test.shape

# model_name = 'bert-base-uncased'

# distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
# training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
# validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
# distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 22
	95percentile : 41
	99percentile : 60


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 22
	95percentile : 39
	99percentile : 58


In [13]:
MAXLEN = 150

X = training_df['sentence']
y = training_df['label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18, stratify=y)


model_name = 'bert-base-uncased'

distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 56


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 22
	95percentile : 42
	99percentile : 57


In [14]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights

class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = dict(zip(classes, class_weights))

[0.66833919 1.98509687]


In [ ]:
# Build BERT model
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
distillbert_learner.set_weight_decay(0.001)
distillbert_learner.autofit(3.328132762062889e-05, epochs=12, early_stopping=4, class_weight=class_weights)



begin training using triangular learning rate policy with max lr of 3.328132762062889e-05...
Epoch 1/12
400/400 [==============================] - 160s 354ms/step - loss: 0.3457 - accuracy: 0.8302 - val_loss: 0.1313 - val_accuracy: 0.9663
Epoch 2/12
400/400 [==============================] - 157s 390ms/step - loss: 0.0680 - accuracy: 0.9871 - val_loss: 0.0615 - val_accuracy: 0.9850
Epoch 3/12
400/400 [==============================] - 144s 358ms/step - loss: 0.0287 - accuracy: 0.9950 - val_loss: 0.0659 - val_accuracy: 0.9850
Epoch 4/12
400/400 [==============================] - 153s 380ms/step - loss: 0.0163 - accuracy: 0.9975 - val_loss: 0.0367 - val_accuracy: 0.9888
Epoch 5/12
141/400 [=========>....................] - ETA: 1:41 - loss: 0.0113 - accuracy: 0.9976

In [29]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

9/9 [==============================] - 13s 678ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       200
           1       1.00      0.99      0.99        67

    accuracy                           1.00       267
   macro avg       1.00      0.99      0.99       267
weighted avg       1.00      1.00      1.00       267



array([[200,   0],
       [  1,  66]])

In [30]:
distillbert_learner.model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [32]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [33]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [34]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [35]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 219, False Positive: 2, False Negative: 1, True Positive: 74


In [36]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       221
           1       0.97      0.99      0.98        75

    accuracy                           0.99       296
   macro avg       0.98      0.99      0.99       296
weighted avg       0.99      0.99      0.99       296
 



In [37]:
training_df

,sentence,label,phrase
0,i chose to take sci 230 to have a larger community around me so i could have more structure and help with bio 230.,1,['I chose to take Sci 230 to have a larger community around me so I could have more structure and help with Bio 230.']
1,"other than that, i don't really know why else i'm here.",0,"[""I'm here to help people, whether it means medically or emotionally, I just want to help. I realized that I actually really love helping others because I care a lot and sometimes I don't even realize that I am. Speaking as myself without any occupation of any sort, I just want to love others and make them feel like they're worth a lot.""]"
2,"the school is a good size, and has a lot of amazing buildings.",0,"['I like all the clubs the school has, the programs, and the internships', 'Another reason why I am in San Francisco state because some of my high school friends were also planning to attend the school.']"
3,"going to college isnt only for ones self either, as having a college education will help anyone provide for their potential families they may have some day.",1,['college is a place that you can network and make connections to people from different places to broaden your point of view socially and in some ways politically.']
4,"when they were young, they escaped the genocide known as the khmer rouge in phnom penh, cambodia.",0,"[""I am expected to carry on both cultures to the next generation. There's more of a force rather than a will to continue living this way, mostly it is out of guilt. To be quite honest, this is not the way I want to live. It's a selfish wish, which is why I continue to live for others, to nurture them and attend to their needs.""]"
...,...,...,...
2659,i want to also open up my own non profit organization.,0,"['I want to help people and connect to patients. I want to connect to the individual on a personal level and get to have a better relationship with them. I would love to be able to help people physically and mentally, while being a friend. I would love to work with elders and get to know them, their stories, and experiences.']"
2660,and my goal in this life is to one day go back to my country and open an all women clinic for women to be able to give birth there without their husbands thinking otherwise.,0,"['I want to inspire others and help them have what I never had.', 'One day when I am not on this earth anymore, I want people to remember me for something I did in my life that inspired them.', 'I have always wanted to help people out.']"
2661,i am here to hopefully make a difference in the world and help others.,1,['I am here to hopefully make a difference in the world and help others.']
2662,"all in all, im here so that i can try my best in school and get through my major!",0,"['Also, its nice to have classmates that are also in a similar position to me and also want to do good in math class because they help inspire me to keep going and trying harder. It used to be hard for me to ask for help because I would think that Im supposed to figure out everything by myself, but taking classes like this one helps show me that, help can be a very wonderful thing, and can help me achieve things and grow with others.']"


In [58]:
test_df_copy = test_df.copy()
test_df_copy["predicted_label"] = y_pred_distillbert
test_df_copy

,sentence,label,phrase,predicted_label
0,every time i have to spend more effort and time to understand the physics concept and finish my homework.,0,"['I am an International student from China, and my parents send me here because they want me to have better education resources.']",0
1,there is a big need for people of color in the medical field and i want to start to make a difference.,0,"[""I am also here to advance my education and make my family proud. I'll be the first in my family to graduate from college and to attend college in general but I want to make them proud.""]",0
2,i enjoy the process of breaking down problems to a point that they can be solved.,0,['I decided to follow my parents lead.'],0
3,"furthermore, i am here learning physics because it is not only a requirement for my major, but because i am excited to learn the subject itself and how it applies to our everyday lives.",0,['I am here because I want to honor the sacrifices that both my parents had to take in order for me to get an education here in the United States.'],0
4,this class was a requirement in my major which is why i am here taking it.,0,['I wouldnt be here without my parents who sacrificed a lot for me to be here. I am part of the first generation in my family to go to college and hopefully get a degree and I am trying to make my parents proud but also learn things for myself and get a career that will give me financial stability and also a career where I will help tons of people.'],0
...,...,...,...,...
250,"when i graduated middle school in china, my parents decided send me to the united state to finish my high school and university study.",1,"['When I graduated middle school in China, my parents decided send me to The United State to finish My high school and university study.']",1
251,my aunt is someone i look up to since she is a radiologist at the stanford medical center in palo alto.,0,['My aunt is someone I look up to since she is a radiologist at the Stanford Medical Center in Palo Alto. I want to be able to follow in her steps and to make my family and her proud. Shes someone who highly supports me getting a college education and she is one of the reasons I continue to push myself even when things get difficult.'],0
252,i am here because i care about my community and to help empower those around me.,0,"[""become a role model for my younger cousins back where I'm from.""]",0
253,"in addition to this, i do hope to someday make a difference in the world, and strongly believe that i can do so if i pursue a stem related career.",0,"['I come from an Indian immigrant family, and all of my family members have worked hard to earn their places in the world. Most of my family members are successful, and a majority of them are working in STEM related careers. Due to this, there has always been an expectation for me to go into a STEM related field.']",0


In [59]:
test_df_copy.to_excel("Familial_error_analysis_2.xlsx")

In [24]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [21]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.9306334080717489


In [22]:
from sklearn.metrics import auc, precision_recall_curve

# Replace these with your actual data
# distillbert_test_label: Ground truth labels (list or numpy array)
# y_pred_distillbert: Predicted probabilities (list or numpy array)

# Calculate precision-recall curve
precision, recall, _ = precision_recall_curve(distillbert_test_label, y_pred_distillbert)

# Calculate PR AUC
pr_auc_score = auc(recall, precision)

# Print the result
print("AUC PR score for DistilBERT model: ", pr_auc_score)

AUC PR score for DistilBERT model:  0.7918205047976637
